# Deep Research with AutoGen

This notebook implements a multi-agent deep research system using AutoGen. It creates a team of specialized agents that collaborate to perform in-depth research on a given topic.

In [1]:
import os, nest_asyncio
nest_asyncio.apply()
import asyncio
import timeit
from autogen_ext.models.ollama import OllamaChatCompletionClient
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.tools import TeamTool
# or import
from autogen_core.models import ModelInfo
import yaml

from tools import create_chat_completion_client

In [2]:
# load model info from resources.yaml
with open("resources.yaml", "r") as f:
    resources = yaml.load(f, Loader=yaml.SafeLoader)
model = [m for m in resources["models"] if m.get("default", False)][0]
print("Using model:", model["name"], "type:", model["type"])
    
model_client = create_chat_completion_client(model)

Using model: gpt-4.1-nano type: openai


In [3]:
# Token usage tracking
from token_counting import (
    wrap_chat_client_for_tokens,
    print_token_usage,
    print_token_usage_with_estimate,
    estimate_conversation_tokens,
    log_token_usage,
)

# Wrap existing model_client (created in the previous cell)
model_client = wrap_chat_client_for_tokens(model_client)

In [113]:
# Create specialized agents
agentConfig = {
    "Researcher":  dict(system_message="Gather and summarize factual info."),
    "FactChecker": dict(system_message="Verify facts and cite sources."),
    "Critic": dict(system_message="Critique clarity and logic."),
    "Summarizer": dict(system_message="Apply critic result and condense into a brief executive summary."),
    "Editor": dict(system_message="Do some language polishing on the summarizer result. Signal APPROVED when done."),
}

agents = []
for agent_type, props in agentConfig.items():
    agents.append(AssistantAgent(name=agent_type, **props, model_client=model_client))

# Set up termination conditions
max_msgs = MaxMessageTermination(max_messages=20)
text_term = TextMentionTermination(text="APPROVED", sources=["Editor"])
termination = max_msgs | text_term

# Create the team
team = RoundRobinGroupChat(
    participants=agents,
    termination_condition=termination
)

In [114]:
# Create the host agent with deep dive capabilities
deepdive_tool = TeamTool(team=team, name="DeepDive", description="Collaborative multi-agent deep dive")

host = AssistantAgent(
    name="Host",
    model_client=model_client,
    tools=[deepdive_tool],
    system_message="You have access to a DeepDive tool for in-depth research."
)

In [115]:
async def run_deepdive(topic: str):
    start_time_s = timeit.default_timer()
    result = await host.run(task=f"Deep dive on: {topic}")
    # Try native usage first; fallback to estimation
    print_token_usage_with_estimate(model_client, getattr(result, 'messages', None), agentConfig.keys())
    elapsed = timeit.default_timer()-start_time_s
    print(f"🔍 DeepDive result (took {elapsed:.2f} sec.) resulted in {len(result.messages)} messages.")
    return result  # Return the result for further exploration

# Set your research topic
topic = "Impacts of Model Context Protocol on Agentic AI"

# Run the deep dive
result = await run_deepdive(topic)

Estimated token usage -> prompt: 7582, completion: 7405, total: 14987 (method=heuristic)
🔍 DeepDive result (took 20.99 sec.) resulted in 10 messages.


In [116]:
# Display token usage after the deep dive
print("=== Token Usage Summary ===")
print_token_usage_with_estimate(model_client, getattr(result, 'messages', None))

=== Token Usage Summary ===
Estimated token usage -> prompt: 7582, completion: 7405, total: 14987 (method=heuristic)


## Explore the Results

Now that we have the results, let's explore them. The cells below will help you analyze the output.

In [117]:
# Display the full result structure
print("Result type:", type(result))
print("\nResult attributes:")
print([attr for attr in dir(result) if not attr.startswith('_')])

Result type: <class 'autogen_agentchat.base._task.TaskResult'>

Result attributes:
['construct', 'copy', 'dict', 'from_orm', 'json', 'messages', 'model_computed_fields', 'model_config', 'model_construct', 'model_copy', 'model_dump', 'model_dump_json', 'model_extra', 'model_fields', 'model_fields_set', 'model_json_schema', 'model_parametrized_name', 'model_post_init', 'model_rebuild', 'model_validate', 'model_validate_json', 'model_validate_strings', 'parse_file', 'parse_obj', 'parse_raw', 'schema', 'schema_json', 'stop_reason', 'update_forward_refs', 'validate']


In [118]:
# Clean up resources
await model_client.close()

In [ ]:
import re

print("Stop reason:", result.stop_reason)
last_message = result.messages[-1].content

print("Message Sequence:")
agent_names = list(agentConfig.keys())
pattern = re.compile(r'(' + '|'.join(re.escape(n) for n in agent_names) + r'):')
for match in pattern.finditer(last_message):
    print(" -", match.group(1))

print("\n", last_message)  # Print the content of the last message

Stop reason: None
Message Sequence:
 - Researcher
 - FactChecker
 - Critic
 - Summarizer
 - Editor
Researcher: **In-Depth Analysis of the Impacts of Model Context Protocol on Agentic AI**

---

**Introduction**

The advent of agentic AI—artificial intelligence systems that exhibit autonomous decision-making, goal-oriented behavior, and potentially adaptable reasoning—raises significant questions about how these agents interpret, utilize, and manage their internal knowledge and contextual understanding. The **Model Context Protocol (MCP)**, a conceptual or emerging framework (assuming based on recent AI research developments), aims to structure and improve the way AI models incorporate and maintain contextual information during interaction and reasoning processes.

This analysis explores how MCP influences the behavior, decision-making, and autonomy of AI agents, along with the ethical, technical, and practical implications.

---

### 1. Overview of Model Context Protocol (MCP)

While e

In [ ]:
# Optional: append token usage (estimated or native) to a CSV log for tracking across runs
# log_token_usage imported from token_counting
log_token_usage(topic, model_client, getattr(result, 'messages', None), agentConfig.keys())

Logged token usage to token_usage_log.csv: {'timestamp': '2025-09-11T15:09:45.315639+00:00', 'topic': 'Impacts of Model Context Protocol on Agentic AI', 'prompt': 7582, 'completion': 7405, 'total': 14987, 'method': 'heuristic'}
